# How to use the AetionResultExtractor code

## Opening Jupyter Notebook 

These instructions assume you have Python and Jupyter Notebook installed. 

First, open the terminal on your computer. Copy the folder path where this code and your files are saved. Type the following into the terminal to change directory (cd) to this folder:

``` 
cd "/User/folder_path/where my files are"
``` 

Next, type jupyter notebook into your terminal. This will open an instance of Jupyter Notebook in your web browser.

``` 
jupyter notebook
``` 

Now you should be able to see the contents of the folder you originally copied the path for in a Jupyter instance. Click the 'AetionResultExtractor.ipynb' to open this notebook and begin your results extraction!

## Running the code

Run the below huge block of code, do not edit it.

In [2]:
import pandas as pd
import numpy as np
import math
from openpyxl import load_workbook

def automate_aetion_results_csv(result_type, exposure, referent, folder_path, file_name, print_on):
    file_path = folder_path + file_name
    header_patchar = np.array(['tableId', 'tableType', 'analysisId', 'analysisOutcome', 'analysisType', 'analysisLabel', 'cohortType', 'exposureIndex', 'exposureCategoryType', 'exposureCategory', 'variableName', 'variableLevel', 'n', 'pct', 'mean', 'sd', 'median Q2', 'IQR Q1', 'IQR Q3', 'Min', '5%', '10%', '20%', '25%', '30%', '40%', '50%', '60%', '70%', '75%', '80%', '90%', '95%', 'Max', 'std diff'])
    header_dichot = np.array(['tableId', 'tableType', 'analysisId', 'analysisOutcome', 'analysisType', 'analysisLabel', 'cohortType', 'exposureIndex', 'exposureCategoryType', 'exposureCategory', 'totalOutcome', 'totalPersonYears', 'exposureCount', 'numOutcomes', 'personYears', 'rateRatio', 'riskRatio','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN'])
    header_reg_model = np.array(['tableId', 'tableType', 'analysisId', 'analysisOutcome', 'analysisType', 'analysisLabel', 'cohortType', 'effectType', 'modelName', 'effect', 'effectP', 'effectStandardError', 'notes','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN','NaN'])
    headers = np.array([header_patchar,header_dichot,header_reg_model])
    results_raw = pd.read_csv(file_path)
    results = np.vstack([headers, np.array(results_raw)])
    results = df = pd.DataFrame(results)
    results.columns = results.iloc[1]
    DICHOTOMOUS_EFFECT = results.copy(deep=True)
    results.columns = results.iloc[2]
    REGRESSION_MODELS = results.copy(deep=True)
    results.columns = results.iloc[0]
    PATIENT_CHAR = results.copy(deep=True)   
    
    DICHOTOMOUS_EFFECT = DICHOTOMOUS_EFFECT[DICHOTOMOUS_EFFECT['tableType'] == 'DICHOTOMOUS_EFFECT']
    REGRESSION_MODELS = REGRESSION_MODELS[REGRESSION_MODELS['tableType'] == 'REGRESSION_MODELS']
    PATIENT_CHAR = PATIENT_CHAR[PATIENT_CHAR['tableType'] != 'REGRESSION_MODELS'] 
    PATIENT_CHAR = PATIENT_CHAR[PATIENT_CHAR['tableType'] != 'DICHOTOMOUS_EFFECT']
    PATIENT_CHAR = PATIENT_CHAR[PATIENT_CHAR['tableType'] != 'tableType']
    
    if result_type == "PS":
        rate_csv = DICHOTOMOUS_EFFECT[DICHOTOMOUS_EFFECT['cohortType'] == 'WEIGHTED']
        hr_csv = REGRESSION_MODELS[REGRESSION_MODELS['effectType'] == 'PS_WEIGHTED']
    else:
        rate_csv = DICHOTOMOUS_EFFECT[DICHOTOMOUS_EFFECT['cohortType'] == 'ALL']
        hr_csv = REGRESSION_MODELS[REGRESSION_MODELS['effectType'] == 'UNADJUSTED']
    
    rate_csv = rate_csv.reset_index(drop=True)
    hr_csv = hr_csv.reset_index(drop=True)

    rate_csv['totalOutcome'] = pd.to_numeric(rate_csv['totalOutcome'])
    rate_csv['totalPersonYears'] = pd.to_numeric(rate_csv['totalPersonYears'])
    rate_csv['numOutcomes'] = pd.to_numeric(rate_csv['numOutcomes'])
    rate_csv['personYears'] = pd.to_numeric(rate_csv['personYears'])

    hr_csv['effect'] = pd.to_numeric(hr_csv['effect'])
    hr_csv['effectStandardError'] = pd.to_numeric(hr_csv['effectStandardError'])
    table = []
    for j in range(len(hr_csv)):
        #print(j)
        i = (j*2)+1
        #print(i)
        subgroup_hr = hr_csv['analysisLabel'].iloc[j] 
        outcome_hr = hr_csv['analysisOutcome'].iloc[j]
        subgroup_rate = rate_csv['analysisLabel'].iloc[i] 
        outcome_rate = rate_csv['analysisOutcome'].iloc[i]
        ref_out = rate_csv['numOutcomes'].iloc[i-1] 
        ref_py = rate_csv['personYears'].iloc[i-1]
        exp_out = rate_csv['numOutcomes'].iloc[i] 
        exp_py = rate_csv['personYears'].iloc[i]

        ref_rate = rate_csv['numOutcomes'].iloc[i-1] /  rate_csv['personYears'].iloc[i-1]
        exp_rate = rate_csv['numOutcomes'].iloc[i] /  rate_csv['personYears'].iloc[i]
        p_rate_diff = exp_rate - ref_rate
        se_rate_diff = math.sqrt((rate_csv['numOutcomes'].iloc[i-1] /  ((rate_csv['personYears'].iloc[i-1])**2)) + ((rate_csv['numOutcomes'].iloc[i] /  (rate_csv['personYears'].iloc[i])**2)))
        z_score = 1.96
        lcb_rate = p_rate_diff - z_score* se_rate_diff  #lower limit of the CI
        ucb_rate = p_rate_diff + z_score* se_rate_diff  #upper limit of the CI
        p_HR = hr_csv['effect'].iloc[j] 
        lcb = math.log(p_HR) - z_score* hr_csv['effectStandardError'].iloc[j]  #lower limit of the CI
        ucb = math.log(p_HR) + z_score* hr_csv['effectStandardError'].iloc[j]  #upper limit of the CI
        e_lcb = math.exp(lcb)
        e_ucb = math.exp(ucb)
        HR = "{:.2f}".format(p_HR) + " (" + "{:.2f}".format(e_lcb)  +", "+ "{:.2f}".format(e_ucb) + ")"

        if subgroup_rate == subgroup_hr and outcome_rate == outcome_hr:
            row = [subgroup_rate, outcome_rate,"{:.0f}".format(ref_py), "{:.0f}".format(exp_py),ref_out, exp_out, "{:.2f}".format(ref_rate*1000), "{:.2f}".format(exp_rate*1000), HR]
            table.append(row)
            if print_on:
                print(row)
    df_table = pd.DataFrame(table)
    df_table.columns = ['Subgroup', 'Outcome', referent + ' '+result_type+' PY', exposure +' '+result_type+' PY', referent +' '+result_type+ ' Events', exposure +' '+result_type+' Events', referent + ' '+ result_type+' Rate', exposure +' '+ result_type+' Rate',  result_type+' Hazard Ratio', result_type+' Rate Difference']
    for col in df_table.columns:
        df_table[col] = df_table[col].astype(pd.StringDtype())
    df_table.to_excel(folder_path+exposure+"_vs_"+referent+"_"+result_type+"_Results.xlsx")
    return str(folder_path+exposure+"_vs_"+referent+"_"+result_type+"_Results.xlsx")

def automate_aetion_results_xls(result_type, exposure, referent, folder_path, csv_name, print_on):
    file_path = folder_path + file_name 
    result_key = pd.read_excel(file_path, sheet_name="Result Key", header =0)
    if result_type == "PS":
        # captilization was off did not match what is in excel, fixed.
        model = 'Propensity score matched'
        rate_search = result_key[result_key['Result Name'].str.contains(r'Matched Analysis (Matched on Propensity Score)>Basic Rate Parameters', regex=False)]
    elif result_type == "WEIGHTED":
        # captilization was off did not match what is in excel, fixed.
        model = 'Propensity score weighted'
        rate_search = result_key[result_key['Result Name'].str.contains(r'Weighted Analysis (IPW/ATE Weights)>Weighted Basic Rate Parameters', regex=False)]
    elif result_type == "HDPS":
        model = 'High-dimensional propensity score matched'
        rate_search = result_key[result_key['Result Name'].str.contains(r'Matched Analysis (Matched on High-Dimensional Propensity Score)>Basic Rate Parameters', regex=False)]    
    else:
        result_type = "Crude"
        model = 'Unadjusted'
        rate_search = result_key[result_key['Result Name'].str.contains(r'Treatment Effect Estimates>Basic Rate Parameters', regex=False)]
    hazard_search = result_key[result_key['Result Name'].str.contains(r'Treatment Effect Estimates>Treatment Effects', regex=False)]
    
    rate_tables = rate_search['Tab'].to_list()
    hazard_tables = hazard_search['Tab'].to_list()
    #print(hazard_tables)

    table = []
    for i in range(len(rate_tables)):
        #print(i)
        sheet_rate = pd.read_excel(file_path, sheet_name= rate_tables[i], header =1)
        j=i*2
        k=j+1
        sheet_xu_rate = pd.read_excel(file_path, sheet_name=hazard_tables[k], header =1)
        #print("XU")
        #print(k)
        #print(sheet_xu_rate)
        #print(hazard_tables[k])
        sheet_hr = pd.read_excel(file_path, sheet_name=hazard_tables[j], header =1)
        #print("HR")
        #print(j)
        #print(sheet_hr)
        #print(hazard_tables[j])
        tab_row_rate = result_key[result_key["Tab"] == rate_tables[i]]
        tab_row_hr = result_key[result_key["Tab"] == hazard_tables[j]]
        type_rate_name = tab_row_rate['PopulationType Label'].iloc[0]
        type_hr_name = tab_row_rate['PopulationType Label'].iloc[0]
        subgroup_rate = tab_row_rate['Subgroup Name'].iloc[0] 
        subgroup_hr = tab_row_hr['Subgroup Name'].iloc[0]
        outcome_rate = tab_row_rate['Outcome Name'].iloc[0] 
        outcome_hr = tab_row_rate['Outcome Name'].iloc[0]
        #breakpoint()
        # this writes over the code for CRUDE type analyses for which these weighted measurements dont exist therefore we need two code blocks for this
        ref_n = sheet_rate[sheet_rate['Parameter'] == 'Number of patients'].iloc[0][1]
        exp_n = sheet_rate[sheet_rate['Parameter'] == 'Number of patients'].iloc[0][2]   
        if result_type == "WEIGHTED":
            ref_py = sheet_rate[sheet_rate['Parameter'] == 'Number of person-years (unweighted)'].iloc[0][1]
            exp_py = sheet_rate[sheet_rate['Parameter'] == 'Number of person-years (unweighted)'].iloc[0][2]
            ref_out = sheet_rate[sheet_rate['Parameter'] == 'Number of events'].iloc[0][1]
            exp_out = sheet_rate[sheet_rate['Parameter'] == 'Number of events'].iloc[0][2]
            ref_rate = sheet_rate[sheet_rate['Parameter'] == 'Weighted rate per 1,000 person-years (95% CI)'].iloc[0][1]
            exp_rate = sheet_rate[sheet_rate['Parameter'] == 'Weighted rate per 1,000 person-years (95% CI)'].iloc[0][2]
            
        else:
            ref_py = sheet_rate[sheet_rate['Parameter'] == 'Number of person-years'].iloc[0][1]
            exp_py = sheet_rate[sheet_rate['Parameter'] == 'Number of person-years'].iloc[0][2]
            ref_out = sheet_rate[sheet_rate['Parameter'] == 'Number of events'].iloc[0][1]
            exp_out = sheet_rate[sheet_rate['Parameter'] == 'Number of events'].iloc[0][2]
            ref_rate = sheet_rate[sheet_rate['Parameter'] == 'Rate per 1,000 person-years'].iloc[0][1]
            exp_rate = sheet_rate[sheet_rate['Parameter'] == 'Rate per 1,000 person-years'].iloc[0][2]
            rate_diff = sheet_rate[sheet_rate['Parameter'] == 'Rate difference per 1,000 person-years (vs. referent; 95% CI)'].iloc[0][2]      
            # xu_rate_diff = sheet_xu_rate[sheet_xu_rate['Model'] == model]['Estimate (per 1000 Person-years)'].iloc[0]
        #if sheet_hr.columns[1] =='Estimate (per 1000 Person-years)':
        #    sheet_hr = pd.read_excel(file_path, sheet_name=hazard_tables[i+1], header =1)
        #    PS_HR = sheet_hr[sheet_hr['Model'] == model]['Hazard Ratio'].iloc[0]
        #else:
        try:
            PS_HR = sheet_hr[sheet_hr['Model'] == model]['Hazard Ratio'].iloc[0]
        except:
            PS_HR = "NA"
            
        if type_rate_name == type_hr_name:
            if result_type == "WEIGHTED":
                row = [subgroup_rate, outcome_rate,ref_n, exp_n, ref_py, exp_py, ref_out, exp_out, ref_rate, exp_rate, PS_HR]
            else:
                row = [subgroup_rate, outcome_rate,ref_n, exp_n, ref_py, exp_py, ref_out, exp_out, ref_rate, exp_rate, PS_HR, rate_diff]
            #row = [subgroup_rate, outcome_rate,"{:.0f}".format(ref_py), "{:.0f}".format(exp_py),ref_out, exp_out, "{:.2f}".format(ref_rate), "{:.2f}".format(exp_rate), PS_HR]
            table.append(row)
            if print_on:
                print(row)
    df_table = pd.DataFrame(table)
    if result_type == "WEIGHTED":
        df_table.columns = ['Subgroup', 'Outcome', referent + ' '+result_type+' N', exposure +' '+result_type+' N', referent + ' '+result_type+' PY', exposure +' '+result_type+' PY', referent +' '+result_type+ ' Events', exposure +' '+result_type+' Events', referent + ' '+ result_type+' Rate', exposure +' '+ result_type+' Rate',  result_type+' Hazard Ratio' ]#, result_type+' Rate Difference']
    else:
        df_table.columns = ['Subgroup', 'Outcome', referent + ' '+result_type+' N', exposure +' '+result_type+' N', referent + ' '+result_type+' PY', exposure +' '+result_type+' PY', referent +' '+result_type+ ' Events', exposure +' '+result_type+' Events', referent + ' '+ result_type+' Rate', exposure +' '+ result_type+' Rate',  result_type+' Hazard Ratio' , result_type+' Rate Difference']
    for col in df_table.columns:
        df_table[col] = df_table[col].astype(pd.StringDtype())
    df_table.to_excel(folder_path+exposure+"_vs_"+referent+"_"+result_type+"_Results.xlsx")
    return str(folder_path+exposure+"_vs_"+referent+"_"+result_type+"_Results.xlsx")

def automate_aetion_results(aetion_version, result_type, exposure, referent, folder_path, file_name, print_on):
    if aetion_version == "CSV":
        result_file_name = automate_aetion_results_csv(result_type, exposure, referent, folder_path, file_name, print_on)
    else:
        result_file_name = automate_aetion_results_xls(result_type, exposure, referent, folder_path, file_name, print_on)
    print("Output completed. Saved as: "+result_file_name)

def prompt_inputs():
    aetion_version = ""  # "CSV" or "XLS"
    result_type = ""    # "Crude" or "PS"
    exposure = "" # Type Exposure Name Here
    referent = "" # Type Reference Name Here
    folder_path = " " # Type Folder Path
    file_name = "" # Type File Name ".csv" or ".xls" types
    print_on = ""
    while (aetion_version != "CSV") and (aetion_version != "XLS"):
        aetion_version = input("Enter aetion_version 'CSV' or 'XLS': ")  # "CSV" or "XLS"
    while (result_type != "Crude") and (result_type != "PS"):
        result_type = input("Enter result_type 'Crude' or 'PS': ")  # "Crude" or "PS"
    while exposure == "":
        exposure = input("Enter Exposure Name Here: ")
    while referent == "":
        referent = input("Enter Reference Name Here: ")
    while folder_path == " ":
        print("If program is ran in same folder as result file, press enter.")
        folder_path = input("Enter Folder Path Here: ")
    while file_name == "":
        file_name = input("Enter File Name with '.csv' or '.xls' ending: ")
    while (print_on != "On") and (print_on != "Off"):
        print("About print_on option:")
        print("'On': To see your results as they are generated") 
        print("'Off': To suppress the print statements (runs faster)")
        print_on = input("Enter 'On' or 'Off' for print option: " )
    print("--------------------------------------------")
    print("Aetion Result File Version: "+aetion_version)
    print("Result Type: "+result_type)
    print("Exposure: " + exposure)
    print("Reference: " + referent)
    print("Print Option: " + print_on)
    if print_on == "On":
        print_on = True
    else:
        print_on = False
    print("This will be your file path that the program will try to open the file from:")
    file_path = folder_path + file_name 
    print(file_path)
    correct = input("Are these all options correct? Y/N: ")
    if correct.lower() != "Y".lower():
        prompt_inputs()
    print("------------- Program Running -------------")
    return aetion_version, result_type, exposure, referent, folder_path, file_name, print_on


def automate_aetion_results_prompted():
    aetion_version, result_type, exposure, referent, folder_path, file_name, print_on = prompt_inputs()
    automate_aetion_results(aetion_version, result_type, exposure, referent, folder_path, file_name, print_on)


def auto_check_std_diff_conv(folder_path, file_name, std_diff_level, xu_analysis):
    file_path = folder_path + file_name
    result_key = pd.read_excel(file_path, sheet_name="Result Key", header =0)
    search_ps_model = result_key[result_key['Result Name'].str.contains(r'PS Model', regex=False)]
    search_cox = result_key[result_key['Result Name'].str.contains(r'Cox Regression Diagnostics', regex=False)]
    if xu_analysis:
        search_xu = result_key[result_key['Result Name'].str.contains(r'Xu Regression Diagnostics', regex=False)]
        xu_tables = search_xu['Tab'].to_list()
    search_confounder_diff = result_key[result_key['ResultComponentType'].str.contains(r'CONFOUNDER_DIFFERENCE', regex=False)]
    ps_model_tables = search_ps_model['Tab'].to_list()
    cox_tables = search_cox['Tab'].to_list()
    conf_diff_tables = search_confounder_diff['Tab'].to_list()
    subgroup_list = search_confounder_diff['Subgroup Name'].to_list()
    outcome_list = search_confounder_diff['Outcome Name'].to_list()
    result_list = search_confounder_diff['Result Name'].to_list()
    table = []
    for i in range(len(conf_diff_tables)):
        #print(conf_diff_tables[i])
        conf_diff_sheet = pd.read_excel(file_path, sheet_name= conf_diff_tables[i], header =1)
        conf_diff_sheet = conf_diff_sheet[conf_diff_sheet["Abs. Std. Diff. (Matched)"] != "-"]
        conf_diff_errors = conf_diff_sheet[conf_diff_sheet["Abs. Std. Diff. (Matched)"] > std_diff_level]
        if conf_diff_errors.empty:
            conf_diff_status = 0
            conf_diff_variables = "NA"
        else:
            conf_diff_variables = str(list(conf_diff_errors['Variable']))
            conf_diff_status = len(conf_diff_errors['Variable'])
        cox_sheet = pd.read_excel(file_path, sheet_name= cox_tables[i], header =1)
        cox_status =  cox_sheet['Value'][1]
        if xu_analysis:
            xu_sheet = pd.read_excel(file_path, sheet_name= xu_tables[i], header =1)
            xu_status = xu_sheet['Value'][1]
        else:
            xu_status = "NA"
        ps_model_sheet = pd.read_excel(file_path, sheet_name= ps_model_tables[i], header =1)
        ps_model_errors = ps_model_sheet[ps_model_sheet["Odds Ratio"].str.contains(r'<|>|∞|N/A')]
        if ps_model_errors.empty:
            ps_model_status = 0
            ps_model_variables = "NA"
            ps_model_odds = "NA"
        else:
            ps_model_variables = str(list(ps_model_errors['Variable']))
            ps_model_odds = str(list(ps_model_errors['Odds Ratio']))
            ps_model_status = len(ps_model_errors['Variable'])
        row = [result_list[i], outcome_list[i], subgroup_list[i], conf_diff_status, conf_diff_variables, cox_status, xu_status, ps_model_status, ps_model_variables, ps_model_odds]
        table.append(row)
    df_table = pd.DataFrame(table)
    df_table.columns = ['RESULT', 'OUTCOME', 'SUBGROUP', "Num Abs. Std. Diff. (Matched) >"+str(std_diff_level), "Variables with >"+str(std_diff_level), "COX Convergence Status", "XU Convergence Status", "PS Model Num Extreme ORs", "PS Model Flagged Variables", "PS Model Flagged ORs"]
    df_table.to_excel(folder_path+"CONVERGENCE_CHECK_"+str(std_diff_level)+"_Results_"+file_name+"x")
    print("Results saved in: " + folder_path)
    return 

### Set a folder path

This is where your current Aetion .xls result files are stored and where you want your final results to be saved to. Make sure that your folder path ends with a '/'

In [3]:
folder_path = "/wherever my file is saved/"

In [19]:
# example
# folder_path = "/Users/someone/Dropbox (Partners HealthCare)/foldername/"

### Name your analysis and the file names that you'll be extracting from

In [12]:
analysis = "CMS_HIV_Primary" # Name your analysis
warf_file = "12345.xls" # file to extract from
other_file = "67890.xls" # file to extract from

## auto_check_std_diff_conv

These function calls will produce a file labeled "CONVERGENCE_CHECK" which will check how many variables have an absolute std. difference greater than the threshold set by the value of std_diff_level (i.e. > 0.1). The True/False flag signifies whether you would like the Xu Regression Diagnostics as well as the Cox Regression Diagnostics.

##### auto_check_std_diff_conv arguments:

folder_path = folder path that file_name is in

file_name = name of file

std_diff_level = threshold to evaluate the absolute std. differences at

xu_analysis = whether or not you ran the Xu analysis and want those regression diagnositcs 

In [17]:
# folder_path = specified above
# warf_file = specified above
# std_diff_level = directly specified in the function call below
# xu_analysis = directly specified in the function call below

In [14]:
auto_check_std_diff_conv(folder_path, warf_file, 0.1, True)

Results saved in: /Users/lg436/Library/Mobile Documents/com~apple~CloudDocs/BWH_DoPE/Claire_python_update/


If it runs successfully, you should see a message output: "Results saved in:" followed by your folder path

## automate_aetion_results

This is the main function call that will extract the desired results. Please see below for argument specifications and be careful to use only the options available.

##### auto_check_std_diff_conv arguments:

*aetion_version* = Options are "XLS" or "CSV" - Must exactly match either of these inputs. This tells the function which file type it will be extracting from. Older versions of Aetion may output as CSV's, but the main file type now is XLS.

*result_type* = Options are  "Crude", "PS", "HDPS", and "WEIGHTED" -  Must exactly match either of these inputs or else you may extract results that you did not intend to. 

*exposure* = A short name for the Exposure group you specified in the Aetion analysis.This is only for labeling results and not used to search through the result file at all.

*referent* = A short name for the Referent group you specified in the Aetion analysis. This is only for labeling results and not used to search through the result file at all.

*folder_path* = This should be the folder path that file_name is stored in - Make sure this ends in a '/'

*file_name* = This should be the name of the file to extract information from - Can be a file you specified above (i.e. warf_file) or the name of the file "12345.xls"

*print_on* = This tells the function whether or not you want to print intermediate outputs to the Jupyter notebook screen.

In [20]:
aetion_version="XLS" # options are CSV or XLS
result_type="Crude" # options are only "Crude", "PS", "HDPS", and "WEIGHTED" - these must be typed EXACTLY as this
exposure="Warfarin" # a label for your exposure group
referent="Apixaban" # a label for your referent group
file_name=warf_file # can be a file you specified above or the file name itself i.e. 12345.xls
print_on=False # whether or not to print intermediate outputs as the function runs to the Jupyter notebook screen

In [7]:
automate_aetion_results(aetion_version, 
                        result_type, 
                        exposure, referent, 
                        folder_path, 
                        file_name, 
                        print_on)

Output completed. Saved as: /Users/lg436/Library/Mobile Documents/com~apple~CloudDocs/BWH_DoPE/Claire_python_update/Warfarin_vs_Apixaban_Crude_Results.xlsx


If it runs successfully, you should see a message output: "Output completed. Saved as: " followed by your folder path

In [ ]:
# aetion_version="XLS" # specified above, no need to re-run unless changing
# result_type="Crude" # we change this below for our "PS" analysis
# exposure="Warfarin" # specified above, no need to re-run unless changing
# referent="Apixaban" # specified above, no need to re-run unless changing
# folder_path = specified above, no need to re-run unless changing
# file_name=warf_file # specified above, no need to re-run unless changing
# print_on=False # specified above, no need to re-run unless changing

In [18]:
result_type="PS" # Note we change this to now get the PS results.                 
automate_aetion_results(aetion_version, 
                        result_type, 
                        exposure, referent, 
                        folder_path, 
                        file_name, 
                        print_on)


Output completed. Saved as: /Users/lg436/Library/Mobile Documents/com~apple~CloudDocs/BWH_DoPE/Claire_python_update/Warfarin_vs_Apixaban_PS_Results.xlsx


In [9]:
result_type="WEIGHTED" # Note we change this to now get the WEIGHTED results.                 
automate_aetion_results(aetion_version, 
                        result_type, 
                        exposure, referent, 
                        folder_path, 
                        file_name, 
                        print_on)


Output completed. Saved as: /Users/lg436/Library/Mobile Documents/com~apple~CloudDocs/BWH_DoPE/Claire_python_update/Warfarin_vs_Apixaban_WEIGHTED_Results.xlsx


### Then if you need to change to another comparator or for a sensitivity analysis, just change your parameters!:

In [ ]:
aetion_version="XLS" # options are CSV or XLS
result_type="Crude" # options are only "Crude", "PS", and "WEIGHTED" - these must be typed EXACTLY as this
exposure="OtherExposure" # a label for your exposure group
referent="OtherReferent" # a label for your referent group
file_name=other_file # example
file_name="67890.xls" # can be a file you specified above or the file name itself i.e. 12345.xls
print_on=False # whether or not to print intermediate outputs as the function runs to the Jupyter notebook screen

In [10]:
automate_aetion_results(aetion_version, 
                        result_type, 
                        exposure, referent, 
                        folder_path, 
                        file_name, 
                        print_on)


Output completed. Saved as: /Users/lg436/Library/Mobile Documents/com~apple~CloudDocs/BWH_DoPE/Claire_python_update/Warfarin_Sens_vs_Apixaban_Crude_Results.xlsx


In [11]:
                        result_type="PS"
automate_aetion_results(aetion_version, 
                        result_type, 
                        exposure, referent, 
                        folder_path, 
                        file_name, 
                        print_on)

Output completed. Saved as: /Users/lg436/Library/Mobile Documents/com~apple~CloudDocs/BWH_DoPE/Claire_python_update/Warfarin_Sens_vs_Apixaban_PS_Results.xlsx


In [12]:
                        result_type="WEIGHTED"
automate_aetion_results(aetion_version, 
                        result_type, 
                        exposure, referent, 
                        folder_path, 
                        file_name, 
                        print_on)

Output completed. Saved as: /Users/lg436/Library/Mobile Documents/com~apple~CloudDocs/BWH_DoPE/Claire_python_update/Warfarin_Sens_vs_Apixaban_WEIGHTED_Results.xlsx


# Combining all your output into one Excel

This code can be used to combine all the output files to one excel file with multiple sheets. To edit simply change the .xlsx file names to the order that you want to combine and add names for each sheet you're adding. The first set chunk of lines (pd.read_excel) will read in the specified files and the .to_excel lines will add each file into a new sheet with the specified sheet name.

In [13]:
writer = pd.ExcelWriter(folder_path+analysis+".xlsx")

sheet_1 =pd.read_excel(folder_path+'Warfarin_vs_Apixaban_Crude_Results.xlsx')
sheet_2 =pd.read_excel(folder_path+'Warfarin_vs_Apixaban_PS_Results.xlsx')
sheet_3 =pd.read_excel(folder_path+'Warfarin_vs_Apixaban_WEIGHTED_Results.xlsx')
sheet_4 =pd.read_excel(folder_path+'OtherExposure_vs_OtherReferent_Crude_Results.xlsx')
sheet_5 =pd.read_excel(folder_path+'OtherExposure_vs_OtherReferent_PS_Results.xlsx')
sheet_6 =pd.read_excel(folder_path+'OtherExposure_vs_OtherReferentn_WEIGHTED_Results.xlsx')

sheet_1.to_excel(writer, sheet_name = 'Warf_vs_Apixaban_Crude', index=False)
sheet_2.to_excel(writer, sheet_name = 'Warf_vs_Apixaban_PS', index=False)
sheet_3.to_excel(writer, sheet_name = 'Warf_vs_Apixaban_Weighted', index=False)
sheet_4.to_excel(writer, sheet_name = 'OtherExposure_vs_OtherReferent_Crude', index=False)
sheet_5.to_excel(writer, sheet_name = 'OtherExposure_vs_OtherReferent_PS', index=False)
sheet_6.to_excel(writer, sheet_name = 'OtherExposure_vs_OtherReferent_Weighted', index=False)

writer.save()
writer.close()

/Users/lg436/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
